In [ ]:
#! pip install fake_useragent

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
import random
from fake_useragent import UserAgent
import time

In [2]:
# URL of page to be scraped
country = "au"

job_title_list = []
job_title_index = []
company_list = []
job_id_list = []
location_list = []

# From here we generate a random user agent
ua = UserAgent()

In [3]:
# Get a list of latest proxies from a free website
res = requests.get('https://www.sslproxies.org/', headers={'User-Agent':'Mozilla/5.0'})
soup = BeautifulSoup(res.text,"lxml")
proxy_list = []
for items in soup.select("#proxylisttable tbody tr"):
    proxy_ip = ':'.join([item.text for item in items.select("td")[:2]])
    proxy_list.append(proxy_ip)

print(proxy_list)

['110.74.199.16:63141', '134.249.165.49:53281', '104.248.48.172:31583', '94.158.165.19:45915', '125.26.99.223:36506', '118.175.240.205:43147', '139.59.124.51:8118', '116.203.23.252:3128', '85.172.39.174:47326', '183.88.219.206:41564', '176.56.107.134:52184', '118.172.181.147:34388', '103.210.120.40:34018', '109.75.42.231:53597', '183.88.224.206:8080', '217.219.61.6:8080', '78.154.167.68:8080', '212.191.132.210:8080', '118.27.26.70:3128', '103.69.235.81:8080', '103.12.197.246:8080', '185.23.131.191:8080', '115.75.2.213:38351', '103.115.164.65:8080', '142.93.245.236:31583', '181.196.145.106:52384', '118.174.232.106:50491', '208.163.39.218:53281', '41.190.147.158:54018', '92.255.232.100:55993', '1.10.186.114:56315', '139.5.71.46:23500', '154.73.53.186:8080', '147.91.111.133:37979', '105.30.250.108:53281', '185.142.67.23:8080', '110.36.232.242:49850', '118.174.220.14:43473', '109.86.219.179:53438', '187.35.140.169:3128', '18.135.32.174:80', '202.131.234.142:39330', '103.109.58.54:8080', '8

In [5]:
# Create a job dictionary
job_dict = [
    "Data+Analyst",
    "Data+Scientist",
    "Data+Engineer",
    "Machine+Learning"
]

# Create random parameters
user_agent = ua.random
header = {"user-agent": str(user_agent)}

# Create random proxy list
proxy = random.choice(proxy_list)
proxy_protocol = {
    "http"  : proxy,
    "https" : proxy
}


# Start the main web scraping
for i, title in enumerate(job_dict):

    print("Starting job search for: ", title)
    # Reset page to 0
    page = 0
    counter = True

    while counter == True:
        
        # search query for Data Analyst roles
        url = f"https://{country}.indeed.com/jobs?q={title}&start={page}"
        print("Current page: ", url)

        # Random time gap
        time_gap = random.randrange(3, 7, 1)
        time.sleep(time_gap)
        
        # Retrieve page with the requests module
        response = requests.get(
                            url,
                            #proxies=proxy_protocol,
                            headers=header
                        )
        
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Retrieve the parent divs for all articles
        results = soup.find_all('div', class_='result')
        
        # For page one, calculate the page number by deviding job counts by 15
        if page == 0:
            job_count = soup.find('div', id='searchCountPages').text.strip()
            job_count = job_count.replace(",", "")
            job_count = int(job_count.split(" ")[3])
            page_count = round(job_count / 15, 0)
            page_range = int(page_count)
            print("Page number: ", int(page_count))

        elif page == page_range*10:
            counter = False


        # loop over results to get article data
        for result in results:
            try:
                # scrape the article header 
                job_title = result.find('a', class_='jobtitle').text.strip()
                job_index = i + 1
                company = result.find('span', class_='company').text.strip()
                job_id = result.get('id')
                location = result.find(class_='location').text
                
                job_title_list.append(job_title)
                job_title_index.append(job_index)
                company_list.append(company)
                location_list.append(location)
                job_id_list.append(job_id)
                
            except:
                pass
        
        # Update page parameter by adding 10
        page += 10

        # Every 10 pages, get random UA
        if page % 100 == 0:
            
            user_agent = ua.random
            header = {"user-agent": str(user_agent)}
            print(f"---------------\n\
                A new user-agent was created:\n\
                {user_agent}\n----------------")

        
print("===================\nScraping completed")

 Mac OS X 10_6_5; ar) AppleWebKit/533.19.4 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4
----------------
Current page:  https://au.indeed.com/jobs?q=Data+Scientist&start=100
Current page:  https://au.indeed.com/jobs?q=Data+Scientist&start=110
Current page:  https://au.indeed.com/jobs?q=Data+Scientist&start=120
Current page:  https://au.indeed.com/jobs?q=Data+Scientist&start=130
Current page:  https://au.indeed.com/jobs?q=Data+Scientist&start=140
Current page:  https://au.indeed.com/jobs?q=Data+Scientist&start=150
Starting job search for:  Data+Engineer
Current page:  https://au.indeed.com/jobs?q=Data+Engineer&start=0
Page number:  170
Current page:  https://au.indeed.com/jobs?q=Data+Engineer&start=10
Current page:  https://au.indeed.com/jobs?q=Data+Engineer&start=20
Current page:  https://au.indeed.com/jobs?q=Data+Engineer&start=30
Current page:  https://au.indeed.com/jobs?q=Data+Engineer&start=40
Current page:  https://au.indeed.com/jobs?q=Data+Engineer&start=50
Current page:  ht

In [6]:
#putting list into dataframe

AU_jobmarket = {"Job ID" : job_id_list,
                "Job Title Index" : job_title_index,
                "Job Title" : job_title_list, 
                "Company Name" : company_list, 
                "Company Location" : location_list}

In [11]:
AU_jobmarket_df = pd.DataFrame(AU_jobmarket)
AU_jobmarket_df["Job Title Index"] = AU_jobmarket_df["Job Title Index"]-1
AU_jobmarket_df

,Job ID,Job Title Index,Job Title,Company Name,Company Location
0,p_1b5ec237a774cff5,1,Data Governance Analyst,Interactive Pty Ltd,Sydney NSW
1,p_447bbc295950e8b9,1,Data Reporting Analyst,Best Friends,Altona North VIC
2,p_9131dc14d50f52f5,1,Data Analyst,Gallagher,East Melbourne VIC
3,p_8c9274d0d7445e35,1,Data Analyst,Department of Justice and Community Safety,Melbourne City Centre VIC
4,p_ddea2ccd3791ed55,1,Data Analyst and Econometrician - APS6,First Grade Group,Canberra ACT
...,...,...,...,...,...
4863,p_d1d5810b6f4453b4,4,Senior FullStack Developer - PHP/React,Credit Sense,Melbourne VIC
4864,p_eaa05cd6ac636e6e,4,Senior FullStack Developer - PHP/React,Credit Sense Australia,Sydney NSW
4865,p_f5454739ac822f73,4,Software Engineer (PHP / Node.js / React),Lightspeed POS,Chippendale NSW
4866,p_10795a71a69219c9,4,Senior Full Stack Developer - PHP/React,Credit Sense Australia,Brisbane QLD


In [12]:
AU_jobmarket_df.to_csv("../Clean Data/AU-JobMarket.csv")